In [ ]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

import graphviper

# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

## Download dataset

global (EVN+VLBA) VLBI observation with:
- 5 scans 
- 2 fields 
- 2 spw

In [ ]:
graphviper.utils.data.download("global_vlbi_gg084b_reduced.ms")

## Convert to Processing Set

In [ ]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "global_vlbi_gg084b_reduced.ms"
partition_scheme = "ddi_intent_field"
# partition_scheme="ddi_state"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "global_vlbi_gg084b_reduced.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    partition_scheme=partition_scheme,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

## Processing Set

In [ ]:
ps_name = outfile

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_name=outfile)
ps.summary()

In [ ]:
ps.keys()

In [ ]:
ps['global_vlbi_gg084b_reduced_ddi_1_intent_None_field_id_1']

In [ ]:
ps['global_vlbi_gg084b_reduced_ddi_1_intent_None_field_id_1'].attrs['antenna_xds']

In [ ]:
ps['global_vlbi_gg084b_reduced_ddi_1_intent_None_field_id_1'].attrs['antenna_xds'].name.values

In [ ]:
ps['global_vlbi_gg084b_reduced_ddi_1_intent_None_field_id_1'].baseline_antenna1_id.values

In [ ]:
ps['global_vlbi_gg084b_reduced_ddi_1_intent_None_field_id_1'].baseline_antenna2_id.values